In [1]:
%matplotlib tk
import auto

In [2]:
# Integrate for stable equilibrium to start continuation from there
integration = auto.run(e='drosophila', c='integrate')
init = integration()[-1]  # last point

gcc -g -O2 -ID:\auto\07p/include -O -c drosophila.c -o drosophila.o
gfortran -LD:\auto\07p\lib  -O drosophila.o -o drosophila.exe D:\auto\07p\lib\*.o -lauto_c
Starting drosophila ...

  BR    PT  TY  LAB       I           MIN V         gleak         Eleak          freq          Iks           Ikf           INa      
   1     1  EP    1  -1.20000E+01  -5.45614E+01   6.80000E+00  -5.50000E+01      Infinity   1.83423E-01   6.90793E-07  -1.49163E+00
   1     1        2  -1.20000E+01  -5.45614E+01   6.80000E+00  -5.50000E+01      Infinity   1.83423E-01   7.19371E-07  -1.49163E+00
   1     2        3  -1.20000E+01  -5.45614E+01   6.80000E+00  -5.50000E+01      Infinity   1.83423E-01   7.19376E-07  -1.49163E+00
   1     3        4  -1.20000E+01  -5.45614E+01   6.80000E+00  -5.50000E+01      Infinity   1.83423E-01   7.19376E-07  -1.49163E+00
   1     4        5  -1.20000E+01  -5.45614E+01   6.80000E+00  -5.50000E+01      Infinity   1.83423E-01   7.19376E-07  -1.49163E+00
   1     5  EP    6  -1

In [ ]:
zip(init.coordnames, init.coordarray)

In [3]:
# Continue equilibrium forwards and backwards in I from init point
equilibrium = auto.run(init, c='equilibrium')
equilibrium += auto.run(init, c='equilibrium', DS='-')
equilibrium = auto.relabel(auto.merge(equilibrium))  # get single curve with unique labels

Starting drosophila ...

  BR    PT  TY  LAB       I        V                  V            mKs          PERIOD        gleak         Eleak          freq     
   1   651  LP    7  -1.90176E+00  -4.77435E+01  -4.77435E+01   1.47724E-01   3.69210E+04   6.80000E+00  -5.50000E+01   2.70848E-05
   1  1000        8  -4.94013E+00  -4.17429E+01  -4.17429E+01   1.89821E-01   3.69210E+04   6.80000E+00  -5.50000E+01   2.70848E-05
   1  1626  LP    9  -1.48988E+01  -3.45427E+01  -3.45427E+01   2.51706E-01   1.54590E+04   6.80000E+00  -5.50000E+01   6.46873E-05
   1  2000       10  -8.45198E+00  -3.14397E+01  -3.14397E+01   2.82177E-01   3.77712E+01   6.80000E+00  -5.50000E+01   2.64752E-02
   1  3000       11   1.13559E+01  -2.87468E+01  -2.87468E+01   3.10361E-01   1.20454E+01   6.80000E+00  -5.50000E+01   8.30195E-02
   1  4000       12   3.12875E+01  -2.71025E+01  -2.71025E+01   3.28308E-01   9.12536E+00   6.80000E+00  -5.50000E+01   1.09585E-01
   1  5000       13   5.12453E+01  -2.58072E+01  -

D:\Python\27_32bit\lib\site-packages\numpy\core\fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [ ]:
print equilibrium.summary()
auto.plot(equilibrium, stability=True, use_labels=True)

In [4]:
# Continue limit cycle starting at Hopf bifurcation found above
tube = auto.run(equilibrium('HB1'), c='tube')

Starting drosophila ...

  BR    PT  TY  LAB       I           MIN V         MAX V        MAX mKs        PERIOD        gleak         Eleak          freq     
   9  1000       11   4.08777E+01  -3.23705E+01  -1.77637E+01   3.52584E-01   7.45431E+00   6.80000E+00  -5.50000E+01   1.34151E-01
   9  2000       12   2.12608E+01  -3.63629E+01  -1.29058E+01   3.39307E-01   8.67572E+00   6.80000E+00  -5.50000E+01   1.15264E-01
   9  3000       13   2.44370E+00  -4.13294E+01  -6.30433E+00   3.25499E-01   1.56343E+01   6.80000E+00  -5.50000E+01   6.39619E-02
   9  3591  UZ   14  -2.70771E+00  -4.45008E+01  -6.24261E+00   3.21513E-01   1.00000E+02   6.80000E+00  -5.50000E+01   1.00000E-02

 Total Time    0.274E+03
drosophila ... done


In [ ]:
auto.plot(equilibrium + tube, bifurcation_x=['I'], bifurcation_y=['V', 'MAX V', 'MIN V'], stability=True, use_labels=False)

In [ ]:
# Continue saddle-node bifurcation (LP1) for changing gleak (2-parameter continuation)
saddle_node = auto.run(equilibrium('LP1'), c='2par_gleak')
saddle_node += auto.run(equilibrium('LP1'), c='2par_gleak', DS='-')
saddle_node = auto.relabel(auto.merge(saddle_node))

In [ ]:
auto.plot(saddle_node, bifurcation_x='I', bifurcation_y='gleak', use_labels=False)

In [ ]:
# Continue Hopf bifurcation (HB1) for changing gleak (2-parameter continuation)
hopf = auto.run(equilibrium('HB1'), c='2par_gleak', UZSTOP = {'I': [-20, 170]})
hopf += auto.run(equilibrium('HB1'), c='2par_gleak', DS='-')
hopf = auto.relabel(auto.merge(hopf))

In [ ]:
auto.plot(hopf, bifurcation_x='I', bifurcation_y='gleak', use_labels=False)

In [ ]:
# As can be seen from the Hopf continuation above, at gleak=10 Hopf bifurcations dominate the dynamics
# To see them in the (1-parameter) bifurcation diagram:
# - set gleak_0=10. in drosophila.h (AUTO does sometimes not pick this up; delete the drosophila.o file in this case)
# - set UZSTOP = {'I': [-20, 140], ...} in c.equilibrium and c.tube
# - set ILP=1 in c.tube (to detect folds)
# - run the first few cells of this notebook (integration, equilibrium, tube)